# Drug screening dataset

In [16]:
import synapseclient
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns

In [9]:
syn = synapseclient.Synapse()
syn.login()

Welcome, johnbachman!



In [20]:
#from Synapse-stored csv
data_path = syn.get("syn20682897").path
data = pd.read_csv(data_path, low_memory=False)   
#from Synapse table
#results = syn.tableQuery("SELECT * FROM syn20556247")
#data = results.asDataFrame()


In [28]:
data.head()

,drug_screen_id,drug_assay_id,experiment_synapse_id,study_synapse_id,funder,model_name,cellosaurus_id,organism_name,drug_name,DT_explorer_internal_id,dosage,dosage_unit,response,response_type,response_unit,model_type,disease_name,disease_efo_id,symptom_name,symptom_efo_id
0,1,syn11373153.17,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,100.000000,uM,99.979123,percent viability,%,cell line,no disease,NaN,no symptom,NaN
1,1,syn11373153.18,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,33.333430,uM,103.789330,percent viability,%,cell line,no disease,NaN,no symptom,NaN
2,1,syn11373153.19,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,11.111175,uM,102.178887,percent viability,%,cell line,no disease,NaN,no symptom,NaN
3,1,syn11373153.20,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,3.703736,uM,104.653527,percent viability,%,cell line,no disease,NaN,no symptom,NaN
4,1,syn11373153.21,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,1.234554,uM,105.312040,percent viability,%,cell line,no disease,NaN,no symptom,NaN


## Summary data

In [97]:
data_path = syn.get("syn20684161").path
data = pd.read_csv(data_path, low_memory=False)   
data.response_type.unique()

array(['AUC_Simpson', 'AUC_Trapezoid', 'IC50_abs', 'Min_viability'],
      dtype=object)

In [98]:
ic50 = data[data.response_type == 'Min_viability']

In [100]:
data.disease_name.unique()

array(['no disease', 'NF1', 'NF2'], dtype=object)

In [101]:
targetspath = syn.get('syn17091507')
readRDS = robjects.r['readRDS']
targets = readRDS(targetspath.path)

In [81]:
targets_filt = (targets
                .query('mean_pchembl > 6')
                .filter(["internal_id", "hugo_gene", "std_name"])
                .drop_duplicates())

In [88]:
targets.head()

,internal_id,hugo_gene,n_quantitative,mean_pchembl,cv,sd,IC50_nM,AC50_nM,EC50_nM,Potency_nM,Ki_nM,Kd_nM,n_qualitative,std_name,total_n,confidence,pchembl_d,pchembl_t,known_selectivity_index
1,3,HTR7,1,8.01,NaN,NaN,NaN,NaN,NaN,NaN,9.7,NaN,-2147483648,CHEMBL2413451,1,-0.229,8.01,10322.90,1.0
2,4,CHRNA4,1,7.54,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,-2147483648,CHEMBL204871,1,-0.229,15.08,4501.97,0.5
3,4,CHRNB2,1,7.54,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,-2147483648,CHEMBL204871,1,-0.229,15.08,4194.75,0.5
4,5,GSK3A,2,7.76,5.285076,0.410122,34.0,NaN,NaN,NaN,9.0,NaN,-2147483648,CHEMBL3582401,2,0.393,7.76,2034.63,1.0
5,6,FAAH,1,9.30,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,-2147483648,CHEMBL2386554,1,-0.229,9.30,6375.76,1.0


In [83]:
data_tgts = ic50.merge(targets, left_on="DT_explorer_internal_id",
                    right_on="internal_id",
                    how="inner")
#.filter(["DT_explorer_internal_id", "hugo_gene"]
#                                       ).assign(foo = 1
#                        ).pivot_table(values = 'foo',
#                                      index = 'DT_explorer_internal_id',
#                                      columns = "hugo_gene", fill_value = 0)"""

In [90]:
data_tgts = ic50.merge(targets, left_on="DT_explorer_internal_id",
                    right_on="internal_id",
                    how="inner")
data_tgts = data_tgts[['disease_name', 'DT_explorer_internal_id',
           'drug_name', 'std_name', 'dosage', 'response_type',
           'response', 'response_unit',
           'hugo_gene', 'mean_pchembl', 'cv', 'sd', 'IC50_nM', 
           'AC50_nM', 'EC50_nM', 'known_selectivity_index']]
data_tgts = data_tgts[(data_tgts.response != 0) & (data_tgts.response != np.inf)]

In [96]:
data_tgts[data_tgts.std_name == 'TEMSIROLIMUS'].iloc[0:50]

,disease_name,DT_explorer_internal_id,drug_name,std_name,dosage,response_type,response,response_unit,hugo_gene,mean_pchembl,cv,sd,IC50_nM,AC50_nM,EC50_nM,known_selectivity_index
75204,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.000780415,46.08294931]",IC50_abs,1.392725e+03,uM,MTOR,5.75,NaN,NaN,1760.0,NaN,NaN,1.0
75205,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.000780415,46.08294931]",IC50_abs,1.392725e+03,uM,BRAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75206,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.000780415,46.08294931]",IC50_abs,6.024084e+01,uM,MTOR,5.75,NaN,NaN,1760.0,NaN,NaN,1.0
75207,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.000780415,46.08294931]",IC50_abs,6.024084e+01,uM,BRAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75208,no disease,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.00078041474654,46.082949308]",IC50_abs,3.092603e+09,uM,MTOR,5.75,NaN,NaN,1760.0,NaN,NaN,1.0
75209,no disease,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.00078041474654,46.082949308]",IC50_abs,3.092603e+09,uM,BRAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75210,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[7.8e-10,4.61e-05]",IC50_abs,3.287926e-05,uM,MTOR,5.75,NaN,NaN,1760.0,NaN,NaN,1.0
75211,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[7.8e-10,4.61e-05]",IC50_abs,3.287926e-05,uM,BRAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Notebook for old dataset

In [11]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()


In [12]:
targetspath = syn.get('syn17091507')
readRDS = robjects.r['readRDS']

targets = readRDS(targetspath.path)

In [15]:

targets_filt = (targets
                .query('mean_pchembl > 6')
                .filter(["internal_id", "hugo_gene", "std_name"])
                .drop_duplicates())
    
pd.DataFrame.head(targets_filt)

,internal_id,hugo_gene,std_name
1,3,HTR7,CHEMBL2413451
2,4,CHRNA4,CHEMBL204871
3,4,CHRNB2,CHEMBL204871
4,5,GSK3A,CHEMBL3582401
5,6,FAAH,CHEMBL2386554


In [17]:
drug_data_path = syn.get("syn17462699").path
drug_data = pd.read_csv(drug_data_path, low_memory=False)
                       
pd.DataFrame.head(drug_data)

SynapseHTTPError: 403 Client Error: 
You do not have READ permission for the requested entity, syn17462699.